In [1]:
# for cleaning up after tests
import os
dir = "data/test_dir/"
for f in os.listdir(dir):
    if ".bin" in f:
        os.remove(os.path.join(dir, f))

In [2]:
from banners import *
# import csv_relations
import os
# get all dirs with data and order by date
data_dirs = []
root = "data/"
for dd in os.listdir(root):
    d = os.path.join(root, dd)
    print(d)
    if os.path.isdir(d):
        dir_files = Dir(d, dd)
        for f in os.listdir(d):
            if ".bag" in f:
                dir_files.bag = os.path.join(d, f)
            if ".csv" in f:
                dir_files.clicks = os.path.join(d, f)
        data_dirs.append(dir_files)

print(data_dirs)

TOPICS = ['/current_pose', '/cam0/camera/image_raw', '/cam1/camera/image_raw', '/therm/image_raw_throttle']

processed_data = []

# process each bag
for a in data_dirs:
    cc = CameraCombo(a)
    bag = rosbag.Bag(a.bag)
    for topic, msg, t in bag.read_messages(TOPICS):
        # print(topic)
        # stack 3 images together
        if topic == '/therm/image_raw_throttle':
            # print("thermal")
            cc.flir = msg
        if topic == '/cam1/camera/image_raw':
            # print("rgb")
            cc.rgb = msg
        if topic == '/cam0/camera/image_raw':
            # print("noir")
            cc.noir = msg
        if topic == "/current_pose":
            print("stacking")
            cc.pose = msg.pose.pose
            cc.stack(t)
    processed_data.append(cc)
    cc.done()


    # get pose of stack
    # compute coords(xmin, ymin & xmax, ymax) of each stack


data/test_dir
[Dir(path='data/test_dir', date='test_dir', bag='data/test_dir/2023-01-08-10-27-04.bag', clicks='data/test_dir/Data0.csv')]
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stacking
stackin

In [28]:
# type(processed_data[0].parsed['pose'][0])
print(processed_data[0].parsed['xmax'].max(axis=0))
print(processed_data[0].parsed['xmax'].min(axis=0))
print(processed_data[0].parsed['ymax'].max(axis=0))
print(processed_data[0].parsed['ymax'].min(axis=0))
# data_dirs[0].clicks

4438182.790626198
-4119098.1564454213
4103407.1220381507
-6421486.196606415


In [38]:
print(processed_data[0].parsed.pose[0])
print(processed_data[0].parsed.pose[0])

print(processed_data[0].parsed.pose[10][0, 3])
print(processed_data[0].parsed.pose[10][1, 3])

[[1.e+00 0.e+00 0.e+00 2.e+06]
 [0.e+00 1.e+00 0.e+00 2.e+06]
 [0.e+00 0.e+00 1.e+00 1.e+01]
 [0.e+00 0.e+00 0.e+00 1.e+00]]
[[1.e+00 0.e+00 0.e+00 2.e+06]
 [0.e+00 1.e+00 0.e+00 2.e+06]
 [0.e+00 0.e+00 1.e+00 1.e+01]
 [0.e+00 0.e+00 0.e+00 1.e+00]]
587448.9440954959
4090856.33578408


In [41]:
print(processed_data[0].parsed.pose.map(lambda x: x[0, 3]).min(axis=0))
print(processed_data[0].parsed.pose.map(lambda x: x[0, 3]).max(axis=0))
print(processed_data[0].parsed.pose.map(lambda x: x[1, 3]).min(axis=0))
print(processed_data[0].parsed.pose.map(lambda x: x[1, 3]).max(axis=0))

457825.9800416248
2000000.0
2000000.0
8952286.073743489


In [39]:
import csv
import csv_relations
import pymap3d as pm
import utm
proc_click = []

origin = None

# go over the CSVs
for a in data_dirs:
    C = csv_relations.CSVAugmented(a.path)
    with open(a.clicks) as clicks:
        reader = csv.reader(clicks)
        for line in reader:
            if origin == None:
                origin = [float(line[0]), float(line[1]), float(line[2])]
            # breakdown line
            u = utm.from_latlon(float(line[0]), float(line[1]))
            # u = pm.geodetic2ned(float(line[0]), float(line[1]), float(line[2]), origin[0], origin[1], origin[2])
            pt = [u[0], u[1]]
            print(u)
            health = int(line[-1])
            
            C.jungler(pt, health, processed_data)

    C.done()
    proc_click.append(C)

(584117.9067661016, 4093375.1608469607, 10, 'S')
(584118.2042998668, 4093375.6741610044, 10, 'S')
(584118.8038999178, 4093375.347267711, 10, 'S')
(584119.2275744353, 4093375.7065064255, 10, 'S')
(584120.0235970954, 4093377.112352521, 10, 'S')
(584120.8489805096, 4093377.3535354086, 10, 'S')
(584120.7741473969, 4093377.7189213047, 10, 'S')
(584120.6833861809, 4093377.8955374006, 10, 'S')
(584121.3669207998, 4093378.0798384515, 10, 'S')
(584121.6185231026, 4093378.7369292374, 10, 'S')
(584121.9670526031, 4093378.59615597, 10, 'S')
(584122.5337866908, 4093378.8902459177, 10, 'S')
(584122.3557688752, 4093379.7871583914, 10, 'S')
(584122.3557688752, 4093379.7871583914, 10, 'S')
(584123.3887136598, 4093379.741936661, 10, 'S')
(584123.3887136598, 4093379.741936661, 10, 'S')
(584123.2832166721, 4093380.5064312327, 10, 'S')
(584124.3394477979, 4093380.8053797577, 10, 'S')
(584124.3394477979, 4093380.8053797577, 10, 'S')
(584124.527759145, 4093381.5616957094, 10, 'S')
(584125.5060928067, 4093381

In [46]:
print(proc_click[0].related["x"].max(axis=0) - processed_data[0].parsed.pose.map(lambda x: x[0, 3]).max(axis=0))
print(proc_click[0].related["x"].min(axis=0) - processed_data[0].parsed.pose.map(lambda x: x[0, 3]).min(axis=0))
print(proc_click[0].related["y"].max(axis=0) - processed_data[0].parsed.pose.map(lambda x: x[1, 3]).max(axis=0))
print(proc_click[0].related["y"].min(axis=0) - processed_data[0].parsed.pose.map(lambda x: x[1, 3]).min(axis=0))

-1415811.865989245
126291.52204264252
-4858857.117853156
2093370.2824231028


In [44]:
proc_click[0].related.relevant.map(lambda c: len(c)).max(axis=0)

0